In [2]:
import pandas as pd
import json
import matplotlib.pyplot as plt

import keras 
from keras.models import Sequential
from keras.layers import Dense
from keras import layers
import tensorflow as tf 
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
import os, shutil
import numpy as np
import re
import chardet



# Preprocessing Data

# Detect Encoding

In [61]:
def detect_encoding_for_files(directory):
    encoding_results = {}
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):  # Check if the file is a text file.
            file_path = os.path.join(directory, filename)  # Get the full path of the file.
            encoding = detect_encoding(file_path)
            encoding_results[filename] = encoding
    return encoding_results

def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    return encoding
    
# Example usage:
directory = r"C:\Users\david\DeepLearning\DataSets\Movie_Poster_Metadata\groundtruth"
encoding_results = detect_encoding_for_files(directory)
for filename, encoding in encoding_results.items():
    print(f"{filename}: {encoding}")

1980.txt: ascii
1981.txt: ascii
1982.txt: UTF-16
1983.txt: UTF-16
1984.txt: UTF-16
1985.txt: UTF-16
1986.txt: UTF-16
1987.txt: UTF-16
1988.txt: UTF-16
1989.txt: UTF-16
1990.txt: UTF-16
1991.txt: UTF-16
1992.txt: UTF-16
1993.txt: UTF-16
1994.txt: UTF-16
1995.txt: UTF-16
1996.txt: UTF-16
1997.txt: UTF-16
1998.txt: UTF-16
1999.txt: UTF-16
2000.txt: UTF-16
2001.txt: UTF-16
2002.txt: UTF-16
2003.txt: UTF-16
2004.txt: UTF-16
2005.txt: UTF-16
2006.txt: UTF-16
2007.txt: UTF-16
2008.txt: UTF-16
2009.txt: UTF-16
2010.txt: UTF-16
2011.txt: UTF-16
2012.txt: UTF-16
2013.txt: UTF-16
2014.txt: UTF-16
2015.txt: UTF-16


In [86]:
#Checks the encoding and writes all files to a big string 
def read_text_files(directory):
    file_string = ""  # Initialize an empty string to hold all the text.
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):  # Check if the file is a text file.
            file_path = os.path.join(directory, filename)  # Get the full path of the file.
            
            # Detect encoding of the file
            encoding = detect_encoding(file_path)
            
            # Read the file using the detected encoding
            with open(file_path, 'r', encoding=encoding) as file:
                # Read the file and append its content to the big string
                file_content = file.read()
                file_string += file_content + "\n"
    return file_string


def convert_to_json(string):
    # Split the big string into individual objects
    objects = string.split("}\n{")
    # Join the objects with a comma and newline, and maintain curly braces
    json_string = "},\n{".join(objects)
    # Enclose the whole string with square brackets to make it a valid JSON array
    json_string = f"[{json_string}]"
    return json_string

def fix_json(json_string):
    # Define regex patterns for replacing
    patterns = {
        r'ObjectId\("(.*?)"\)': r'"\1"',  # Matches ObjectId() and replaces it with just the string inside the parenthesis
        r'"h"ttp"(.*)"': r'"http\1"',  # Matches "h"ttp" and replaces it with "http"
        r'"(.*?)" : "(.*?)"(.*?)"(.*?)"':'"\1" : "\2\3\4"',  # Matches "Some Header" : "Some "Value" "..." " and replaces it with "Some header" : "Some Value"
        r'[਍ഀ]':'' # replaces all invalid characters with empty string 
    }

    # Apply regex replacements
    for pattern, replacement in patterns.items():
        json_string = re.sub(pattern, replacement, json_string)

    return json_string


def write_json_to_file(json_string, filename, encoding='utf-16'):
    with open(filename, 'w', encoding=encoding) as file:
        file.write(json_string)

def parse_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-16') as file:  # Ensure the correct encoding
            data = json.load(file)
        return data
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return None
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None

In [87]:
file_string = read_text_files(directory)
print(file_string[:830])

{
  "_id" : ObjectId("56fb860e71a0651b0c306f55"),
  "Language" : "English",
  "Box_office" : "290,158,751",
  "Country" : "USA",
  "Rated" : "PG",
  "imdbID" : "tt0080684",
  "Awards" : "Won 1 Oscar. Another 15 wins & 18 nominations.",
  "Poster" : "h"ttp"://ia.media-imdb.com/images/M/MV5BMjE2MzQwMTgxN15BMl5BanBnXkFtZTcwMDQzNjk2OQ@@._V1_SX300.jpg",
  "Director" : "Irvin Kershner",
  "Released" : "20 Jun 1980",
  "Writer" : "Leigh Brackett (screenplay), Lawrence Kasdan (screenplay), George Lucas (story)",
  "imdbVotes" : "799,579",
  "Runtime" : "124 min",
  "Response" : "True",
  "imdbRating" : "8.8",
  "Title" : "Star "Wars": Episode V - The Empire Strikes Back",
  "Genre" : "Action, Adventure, Fantasy",
  "Actors" : "Mark Hamill, Harrison Ford, Carrie Fisher, Billy Dee Williams",
  "Type" : "movie",
  "Metascore" : "


In [90]:
fixed_format = fix_json(file_string)
json_string = convert_to_json(fixed_format)

#Verifying the },{ format 
json_string = json_string.replace("}\n\n{", "},\n\n{")
# Next, replace the single newline sequence
json_string =  json_string.replace("}\n{", "},\n{")

fileName = r'C:\Users\david\DeepLearning\DataSets\Movie_Poster_Metadata\txt_to_json\txt_json.txt'
write_json_to_file(json_string,fileName )

print(detect_encoding(fileName))



UTF-16


In [91]:
json_data = parse_json_file(fileName)

df = pd.DataFrame(json_data)
display(df)

,_id,Language,Box_office,Country,Rated,imdbID,Awards,Poster,Director,Released,...,imdbVotes,Runtime,Response,imdbRating,,Genre,Actors,Type,Metascore,Title
0,56fb860e71a0651b0c306f55,English,"290,158,751",USA,PG,tt0080684,Won 1 Oscar. Another 15 wins & 18 nominations.,http://ia.media-imdb.com/images/M/MV5BMjE2MzQw...,Irvin Kershner,20 Jun 1980,...,"799,579",124 min,True,8.8,,"Action, Adventure, Fantasy","Mark Hamill, Harrison Ford, Carrie Fisher, Bil...",movie,79,NaN
1,56fb860e71a0651b0c306f56,English,"101,500,000",USA,R,tt0081562,1 nomination.,http://ia.media-imdb.com/images/M/MV5BMTM4OTI3...,Sidney Poitier,12 Dec 1980,...,"17,705",111 min,True,6.8,NaN,"Comedy, Crime","Gene Wilder, Richard Pryor, Georg Stanford Bro...",movie,50,Stir Crazy
2,56fb860e71a0651b0c306f57,English,"83,400,000",USA,PG,tt0080339,Nominated for 1 Golden Globe. Another 2 wins &...,http://ia.media-imdb.com/images/M/MV5BNDU2MjE4...,"Jim Abrahams, David Zucker, Jerry Zucker",02 Jul 1980,...,"154,994",88 min,True,7.8,NaN,Comedy,"Kareem Abdul-Jabbar, Lloyd Bridges, Peter Grav...",movie,N/A,Airplane!
3,56fb860e71a0651b0c306f58,English,None,USA,PG,tt0080377,1 win.,http://ia.media-imdb.com/images/M/MV5BMTc5MTc5...,Buddy Van Horn,17 Dec 1980,...,"12,639",116 min,True,6.0,NaN,"Action, Comedy","Clint Eastwood, Sondra Locke, Geoffrey Lewis, ...",movie,N/A,Any Which Way You Can
4,56fb860e71a0651b0c306f59,English,"69,800,000",USA,R,tt0081375,Nominated for 3 Oscars. Another 1 win & 3 nomi...,http://ia.media-imdb.com/images/M/MV5BMjIyNjkx...,Howard Zieff,10 Oct 1980,...,"17,591",109 min,True,6.1,NaN,"Comedy, War","Goldie Hawn, Eileen Brennan, Armand Assante, R...",movie,N/A,Private Benjamin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8868,570780c771a0650b20736c69,English,"1,711",USA,N/A,tt2912776,N/A,http://ia.media-imdb.com/images/M/MV5BMTk4ODE4...,John D. Hancock,27 Aug 2015,...,9,110 min,True,7.2,NaN,"Drama, Music","Dorothy Tristan, Trish Basinger, Ed Ernstes, E...",movie,N/A,The Looking Glass
8869,570780c771a0650b20736c6a,English,"1,631",Australia,UNRATED,tt2241750,8 wins & 3 nominations.,http://ia.media-imdb.com/images/M/MV5BMTcwNzc2...,Dean Francis,04 Mar 2015,...,547,93 min,True,5.7,NaN,"Drama, Sport","Matt Levett, Maya Stange, Jack Matthews, Harry...",movie,N/A,Drown
8870,570780c771a0650b20736c6b,English,"1,056",USA,N/A,tt4797082,N/A,N/A,Muffie Meyer,30 Oct 2015,...,N/A,64 min,True,N/A,NaN,Documentary,N/A,movie,N/A,Making Rounds
8871,570780c771a0650b20736c6c,English,None,USA,N/A,tt5350540,N/A,N/A,N/A,25 Aug 2015,...,N/A,8 min,True,N/A,NaN,"Documentary, Short","Craig Baumgarten, Tony Jaa, Dolph Lundgren, Ro...",movie,N/A,"Skin Trade: Behind the Action, Fighting You Ca..."
